# Bubble Chart spätester Sonnenaufgang

In [2]:
import geopandas as gpd
import pandas as pd
import json
import consts
from utils import categorize_time_to_half_hour, categorize_time_to_half_hour_numerical, dt_to_time, time_to_time, categorize_time_to_6min_numerical, categorize_time_to_6min
import pyperclip
from pathlib import Path
import datetime

In [3]:
# Load timedata
time_data = json.load(open(consts.PATH_TIMEDATA_HEX, 'r', encoding='utf-8'))

# Load Nuts3
gdf_grid_raw = gpd.read_file(consts.PATH_HEXAGON)

## Extremwerte pro Land

In [5]:
# Serialize Timedata
df_timedata = list(map(lambda x: {
    'nuts_id': x['nuts_id'],

    'sunrise_earliest_current': x['sunrise']['current']['earliest'],
    'sunrise_latest_current': x['sunrise']['current']['latest'],
    'sunset_earliest_current': x['sunset']['current']['earliest'],
    'sunset_latest_current': x['sunset']['current']['latest'],

    'sunrise_earliest_winter': x['sunrise']['winter']['earliest'],
    'sunrise_latest_winter': x['sunrise']['winter']['latest'],
    'sunset_earliest_winter': x['sunset']['winter']['earliest'],
    'sunset_latest_winter': x['sunset']['winter']['latest'],

    'sunrise_earliest_summer': x['sunrise']['summer']['earliest'],
    'sunrise_latest_summer': x['sunrise']['summer']['latest'],
    'sunset_earliest_summer': x['sunset']['summer']['earliest'],
    'sunset_latest_summer': x['sunset']['summer']['latest'],

}, time_data))

df_timedata = pd.DataFrame(df_timedata)

def apply_time(dt):
    return datetime.datetime.strptime(dt, '%H:%M:%S.%f')


# Get properties from above
properties = list(df_timedata.keys())
properties.remove('nuts_id')

# Convert strings to time
for property in properties:
    df_timedata[property] = df_timedata[property].apply(apply_time)

gdf_grid = gdf_grid_raw.merge(df_timedata, left_on='NUTS_ID', right_on='nuts_id')
gdf_grid = gdf_grid.drop(columns=['nuts_id'])

df_grouped = gdf_grid.groupby('country').agg(
    sunrise_earliest_current = ('sunrise_earliest_current', 'min'),
    sunrise_latest_current = ('sunrise_latest_current', 'max'),
    sunset_earliest_current = ('sunset_earliest_current', 'min'),
    sunset_latest_current = ('sunset_latest_current', 'max'),

    sunrise_earliest_summer = ('sunrise_earliest_summer', 'min'),
    sunrise_latest_summer = ('sunrise_latest_summer', 'max'),
    sunset_earliest_summer = ('sunset_earliest_summer', 'min'),
    sunset_latest_summer = ('sunset_latest_summer', 'max'),

    sunrise_earliest_winter = ('sunrise_earliest_winter', 'min'),
    sunrise_latest_winter = ('sunrise_latest_winter', 'max'),
    sunset_earliest_winter = ('sunset_earliest_winter', 'min'),
    sunset_latest_winter = ('sunset_latest_winter', 'max')
).reset_index(drop=False)

df_grouped.sort_values(by='sunrise_latest_current', ascending=False, inplace=True)

df_grouped = df_grouped[~df_grouped['country'].isin(['Unknown'])]

# Format for DataWrapper
for property in properties:
    df_grouped[property] = '2025-01-01 ' + df_grouped[property].dt.strftime('%H:%M')

df_grouped['sunrise_difference_minutes'] = (
    pd.to_datetime(df_grouped['sunrise_latest_current']).dt.time.apply(lambda t: t.hour * 60 + t.minute) -
    pd.to_datetime(df_grouped['sunrise_earliest_current']).dt.time.apply(lambda t: t.hour * 60 + t.minute)
)

#df_grouped.to_clipboard(index=False)
df_grouped = df_grouped.sort_values('sunrise_difference_minutes', ascending=False)

# Remove some...
df_grouped = df_grouped[~df_grouped['country'].isin(['Vatikanstadt', 'Färöer', 'Åland'])]

df_neu = pd.concat([
    df_grouped.head(5),
    df_grouped[df_grouped.country == 'Schweiz'],
    df_grouped[df_grouped.country == 'Deutschland'],
    df_grouped.tail(5)
])
df_neu = df_neu.sort_values('sunrise_earliest_current')
df_neu.to_clipboard(index=False)
df_neu

,country,sunrise_earliest_current,sunrise_latest_current,sunset_earliest_current,sunset_latest_current,sunrise_earliest_summer,sunrise_latest_summer,sunset_earliest_summer,sunset_latest_summer,sunrise_earliest_winter,sunrise_latest_winter,sunset_earliest_winter,sunset_latest_winter,sunrise_difference_minutes
29,Norwegen,2025-01-01 02:54,2025-01-01 13:16,2025-01-01 10:53,2025-01-01 23:36,2025-01-01 02:54,2025-01-01 14:16,2025-01-01 11:53,2025-01-01 23:36,2025-01-01 01:54,2025-01-01 13:16,2025-01-01 10:53,2025-01-01 22:36,622
35,Schweden,2025-01-01 03:07,2025-01-01 12:15,2025-01-01 11:19,2025-01-01 22:44,2025-01-01 03:07,2025-01-01 13:15,2025-01-01 12:19,2025-01-01 22:44,2025-01-01 02:07,2025-01-01 12:15,2025-01-01 11:19,2025-01-01 21:44,548
8,Finnland,2025-01-01 03:15,2025-01-01 12:49,2025-01-01 11:35,2025-01-01 23:08,2025-01-01 03:15,2025-01-01 13:49,2025-01-01 12:35,2025-01-01 23:14,2025-01-01 02:15,2025-01-01 12:49,2025-01-01 11:35,2025-01-01 22:14,574
45,Vereinigtes Königreich,2025-01-01 03:33,2025-01-01 09:38,2025-01-01 14:48,2025-01-01 22:58,2025-01-01 03:33,2025-01-01 10:38,2025-01-01 15:48,2025-01-01 23:12,2025-01-01 02:33,2025-01-01 09:38,2025-01-01 14:48,2025-01-01 22:12,365
7,Estland,2025-01-01 03:51,2025-01-01 09:25,2025-01-01 15:06,2025-01-01 22:47,2025-01-01 03:51,2025-01-01 10:25,2025-01-01 16:06,2025-01-01 22:47,2025-01-01 02:51,2025-01-01 09:25,2025-01-01 15:06,2025-01-01 21:47,334
5,Deutschland,2025-01-01 04:30,2025-01-01 08:50,2025-01-01 15:40,2025-01-01 22:07,2025-01-01 04:30,2025-01-01 09:50,2025-01-01 16:40,2025-01-01 22:07,2025-01-01 03:30,2025-01-01 08:50,2025-01-01 15:40,2025-01-01 21:07,260
28,Nordmazedonien,2025-01-01 04:53,2025-01-01 07:05,2025-01-01 15:57,2025-01-01 20:16,2025-01-01 04:53,2025-01-01 08:05,2025-01-01 16:57,2025-01-01 20:16,2025-01-01 03:53,2025-01-01 07:05,2025-01-01 15:57,2025-01-01 19:16,132
0,Albanien,2025-01-01 05:01,2025-01-01 07:11,2025-01-01 16:05,2025-01-01 20:22,2025-01-01 05:01,2025-01-01 08:11,2025-01-01 17:05,2025-01-01 20:22,2025-01-01 04:01,2025-01-01 07:11,2025-01-01 16:05,2025-01-01 19:22,130
36,Schweiz,2025-01-01 05:23,2025-01-01 08:19,2025-01-01 16:29,2025-01-01 21:32,2025-01-01 05:23,2025-01-01 09:19,2025-01-01 17:29,2025-01-01 21:32,2025-01-01 04:23,2025-01-01 08:19,2025-01-01 16:29,2025-01-01 20:32,176
24,Malta,2025-01-01 05:45,2025-01-01 07:18,2025-01-01 16:47,2025-01-01 20:22,2025-01-01 05:45,2025-01-01 08:13,2025-01-01 17:47,2025-01-01 20:22,2025-01-01 04:45,2025-01-01 07:13,2025-01-01 16:47,2025-01-01 19:22,93


## Alle Zellen exportieren

In [ ]:
# Serialize Timedata
df_timedata = list(map(lambda x: {
    'nuts_id': x['nuts_id'],
    'time': x['sunrise']['current']['latest'],
}, time_data))

df_timedata = pd.DataFrame(df_timedata)

# Convert strings to time
df_timedata['time'] = df_timedata['time'].apply(time_to_time)

def categorize_time_to_2min_numerical(dt):
    return round(dt.hour + dt.minute / 60 + (dt.second / 60) / 60, 2)

df_timedata['time_numerical'] = df_timedata['time'].apply(categorize_time_to_2min_numerical)

gdf_grid = gdf_grid_raw.merge(df_timedata, left_on='NUTS_ID', right_on='nuts_id')
gdf_grid = gdf_grid.drop(columns=['nuts_id'])

gdf_grid['lon'] = round(gdf_grid['geometry'].centroid.x, 2)

# Drop more west than -12
gdf_grid = gdf_grid[gdf_grid['lon'] > -12]

gdf_grid = gdf_grid[['geometry', 'time', 'time_numerical', 'lon', 'timezone']]

gdf_grid.to_file(Path('../export/bubble-data.geojson'))


/var/folders/78/tgbd6qq15fd21h4strxf59mh0000gn/T/ipykernel_19080/77619700.py:20: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_grid['lon'] = round(gdf_grid['geometry'].centroid.x, 2)
